In [1]:
from preprocessing_utils import load_all_files, preprocess_documents
from pathlib import Path
import os
from api import GOOGLE_API_KEY
os.environ["GOOGLE_API_KEY"] = GOOGLE_API_KEY

## Text only

In [ ]:
papers_path = Path("papers/selection")
docs = load_all_files(papers_path)
docs = preprocess_documents(docs) 

Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text discovered. Output will be incomplete.
Rotated text

In [2]:
from langchain_core.prompts import PromptTemplate

template = """You are an assistant for question-answering tasks.
Use the following pieces of retrieved context to answer the question.
If you don't know the answer, just say that you don't know.
Use around three sentences for your answer and keep it concise.
Do not ever mention that you are using the context as source.

Question: {question}

Context:
{context}

Answer:"    
"""
custom_rag_prompt = PromptTemplate.from_template(template)

In [3]:
# from langchain import hub
# from langchain_chroma import Chroma
from qdrant_client import QdrantClient
from langchain_qdrant import QdrantVectorStore
from qdrant_client import models
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_google_genai import GoogleGenerativeAIEmbeddings, ChatGoogleGenerativeAI
from langchain_text_splitters import RecursiveCharacterTextSplitter
from langchain.chains import create_retrieval_chain
from langchain.chains.combine_documents import create_stuff_documents_chain
from langchain_core.prompts import ChatPromptTemplate


# text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
# splits = text_splitter.split_documents(docs)
qdrant_client = QdrantClient(path="papers_db")
# client.create_collection(
#     collection_name="selection",
#     vectors_config=models.VectorParams(
#         size=768,
#         distance=models.Distance.COSINE
#     )

# )
embeddings = GoogleGenerativeAIEmbeddings(model="models/text-embedding-004")
# vectorstore = Chroma.from_documents(
#     documents=splits,
#     embedding=embeddings,
#     persist_directory="selection_db"
#  )
# vectorstore = Chroma(
#     persist_directory="selection_db",
#     embedding_function=embeddings
# )
vectorstore = QdrantVectorStore(
    client=qdrant_client,
    collection_name="selection",
    embedding=embeddings,
    distance=models.Distance.COSINE
)
# vectorstore.add_documents(splits)

# Retrieve and generate using the relevant snippets of the blog.
retriever = vectorstore.as_retriever()
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")
prompt = custom_rag_prompt


def format_docs(docs):
    return "\n\n".join(doc.page_content for doc in docs)


# rag_chain = (
#     {"context": retriever | format_docs, "question": RunnablePassthrough()}
#     | prompt
#     | llm
#     | StrOutputParser()
# )

# rag_chain.invoke("What is Task Decomposition?")
system_prompt = (
    "You are an assistant for question-answering tasks."
    "Use the following pieces of retrieved context to answer the question."
    "If you don't know the answer, just say that you don't know."
    "Keep your answer concise and to the point."
    "Use LaTeX for any math equation."
    # "Use around three sentences for your answer and keep it concise."
    "Do not ever mention that you are using the context as source."
    "Here is context for your answer:"
    "\n\n"
    "{context}"
)

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_prompt),
        ("human", "{input}"),
    ]
)
question_answer_chain = create_stuff_documents_chain(llm, prompt)
rag_chain = create_retrieval_chain(retriever, question_answer_chain)

# rag_chain.invoke({"input": "What kind of material is MAPbI3?"})

In [28]:
response = rag_chain.invoke({"input":
"Give me an example architecture of a graph neural network used to predict physical properties of crystals and describe the internal architecture."
                             })
print(response["context"])
print(response["answer"])

[Document(metadata={'producer': 'iText® 5.3.5 ©2000-2012 1T3XT BVBA (SPRINGER SBM; licensed version)', 'creator': 'Springer', 'creationdate': '2021-08-31T00:12:39+05:30', 'keywords': '', 'crossmarkdomains[1]': 'springer.com', 'moddate': '2021-08-31T07:48:08+02:00', 'subject': 'Communications Materials, doi:10.1038/s43246-021-00194-3', 'doi': '10.1038/s43246-021-00194-3', 'author': 'Jiucheng Cheng', 'crossmarkdomains[2]': 'springerlink.com', 'title': 'A geometric-information-enhanced crystal graph network for predicting properties of materials', 'source': 'papers\\selection\\Cheng et al. - 2021 - A geometric-information-enhanced crystal graph net.pdf', 'total_pages': 11, 'page': 1, 'page_label': '2', '_id': '64a0bfbb3d48419a91198657aff450eb', '_collection_name': 'selection'}, page_content='aggregation process. In material prediction domain, the geometrical Crystal graph deﬁ nition and the introduction of geometric\nstructure information like spatial distance and direction is alsoinforma

## Multimodal

In [6]:
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash-lite")

In [7]:
from langchain.prompts import ChatPromptTemplate, SystemMessagePromptTemplate, HumanMessagePromptTemplate
from langchain.schema import HumanMessage, SystemMessage
import base64

# Function to encode an image as base64
def encode_image(image_path):
    with open(image_path, "rb") as img_file:
        return base64.b64encode(img_file.read()).decode("utf-8")

# Encode the image(s)
image_path = "image.jpg"  # Replace with your image path
image_base64 = encode_image(image_path)

# Define system message (instructions for the AI)
system_template = "You are an AI assistant that analyzes images and answers user queries."
system_message = SystemMessagePromptTemplate.from_template(system_template)

# Define human message (text + image context)
human_template = "{text}"  # Placeholder for user input text
human_message = HumanMessagePromptTemplate.from_template(human_template)

# Create the chat prompt template
chat_prompt = ChatPromptTemplate.from_messages([system_message, human_message])

# Format the chat prompt with user input
formatted_prompt = chat_prompt.format_messages(
    text="What can you tell me about this image?",
)

# Append image to the human message dynamically
formatted_prompt.append(
    HumanMessage(
        content=[
            {"type": "text", "text": "Here is the image context:"},
            {"type": "image_url", "image_url": f"data:image/jpeg;base64,{image_base64}"}
        ]
    )
)

# Initialize Gemini model (multimodal)
llm = ChatGoogleGenerativeAI(model="gemini-2.0-flash")

# Get the response from the model
response = llm(formatted_prompt)
print(response.content)

/var/folders/9w/3l6rxjcs3bgfl3v7b2f66llr0000gn/T/ipykernel_77878/2457562682.py:44: LangChainDeprecationWarning: The method `BaseChatModel.__call__` was deprecated in langchain-core 0.1.7 and will be removed in 1.0. Use :meth:`~invoke` instead.
  response = llm(formatted_prompt)


The image features a tabby cat wearing orange sunglasses. The cat has green eyes and a white chest. It's sitting on what appears to be a sofa, with various cushions visible in the background. The image is well-lit and has a shallow depth of field, focusing attention on the cat's face.


In [ ]:
from langchain_core.runnables import RunnableLambda

top_k = 2

def retrieve_images(query):
    return [encode_image("image.jpg") for i in range(top_k)]

def print_thing(thing):
    print(thing)
    return thing

def expand_context_images(message):
    new_message = {}
    new_message["query"] = message["query"]
    for i, image in enumerate(message["context_images"]):
        new_message[f"context{i}"] = image
    
    return new_message

system_message = (
    "You are an assistant for question-answering tasks."
    "Use the images of retrieved context to answer the question."
    "If you don't know the answer, just say that you don't know."
    "Use around three sentences for your answer and keep it concise."
    "Do not ever mention that you are using the context as source."
) 

user_message = [
    {
        "type": "text",
        "text": "{query}",
    }
] + [
    {
        "type": "image_url",
        "image_url": {"url": "data:image/jpeg;base64,{placeholder}".replace("placeholder", f"context{i}")},
    }
    for i in range(top_k)
]

prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system_message),
        (
            "user",
            user_message,
        ),
    ]
)
# prompt = ChatPromptTemplate.from_messages(
#     [
#         ("system", system_message),
#         (
#             "user",
#             [
#                 {
#                     "type": "text",
#                     "text": "{query}",

#                 },
#                 {
#                     "type": "image_url",
#                     "image_url": {"url": "data:image/jpeg;base64,"},
#                 }
#             ],
#         ),
#     ]
# )

rag_chain = (
    {"context_images": retrieve_images, "query": RunnablePassthrough()}
    | RunnableLambda(expand_context_images)
    | prompt
    | llm
    | StrOutputParser()
)

rag_chain.invoke("What are these images? Are they the same?")

'The images show a cat wearing orange sunglasses. Yes, they are the same.'

In [42]:
prompt

ChatPromptTemplate(input_variables=['context0', 'context1', 'query'], input_types={}, partial_variables={}, messages=[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], input_types={}, partial_variables={}, template="You are an assistant for question-answering tasks.Use the images of retrieved context to answer the question.If you don't know the answer, just say that you don't know.Use around three sentences for your answer and keep it concise.Do not ever mention that you are using the context as source."), additional_kwargs={}), HumanMessagePromptTemplate(prompt=[PromptTemplate(input_variables=['query'], input_types={}, partial_variables={}, template='{query}'), ImagePromptTemplate(input_variables=['context0'], input_types={}, partial_variables={}, template={'url': 'data:image/jpeg;base64,{context0}'}), ImagePromptTemplate(input_variables=['context1'], input_types={}, partial_variables={}, template={'url': 'data:image/jpeg;base64,{context1}'})], additional_kwargs={}

## Evaluation

In [4]:
# Only run this block for Gemini Developer API
from google import genai
genai_client = genai.Client(api_key=GOOGLE_API_KEY)

In [5]:
from preprocessing_utils import load_queries, load_answers

queries = load_queries("papers/selection_queries.txt")
answers = load_answers("papers/selection_answers.txt")

In [6]:
from deepeval import evaluate
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.test_case import LLMTestCase
from deepeval_utils import DeepevalGeminiModel

# Replace this with the actual output from your LLM application
actual_output = "I like trains."
llm_judge = DeepevalGeminiModel(genai_client, model_name="gemini-2.0-flash")

metric = AnswerRelevancyMetric(
    threshold=0.7,
    model=llm_judge,
    include_reason=True
)
test_case = LLMTestCase(
    input="What if these shoes don't fit?",
    actual_output=actual_output
)

# To run metric as a standalone
# metric.measure(test_case)
# print(metric.score, metric.reason)

result = evaluate(test_cases=[test_case], metrics=[metric])

✨ You're running DeepEval's latest Answer Relevancy Metric! (using gemini-2.0-flash, strict=False, 
async_mode=True)...

Evaluating 1 test case(s) in parallel: |██████████|100% (1/1) [Time Taken: 00:01,  2.00s/test case]



Metrics Summary

  - ❌ Answer Relevancy (score: 0.0, threshold: 0.7, strict: False, evaluation model: gemini-2.0-flash, reason: The score is 0.00 because the statement in the actual output is completely irrelevant to the input question about shoe fit., error: None)

For test case:

  - input: What if these shoes don't fit?
  - actual output: I like trains.
  - expected output: None
  - context: None
  - retrieval context: None


Overall Metric Pass Rates

Answer Relevancy: 0.00% pass rate




✓ Tests finished 🎉! Run 'deepeval login' to save and analyze evaluation results on Confident AI.
 
✨👀 Looking for a place for your LLM test data to live 🏡❤️ ? Use Confident AI to get & share testing reports, 
experiment with models/prompts, and catch regressions for your LLM system. Just run 'deepeval login' in the CLI.

In [ ]:
from deepeval.dataset import EvaluationDataset, Golden

test_cases = []
for doc_queries, doc_answers in zip(queries, answers):
    for query, answer in zip(doc_queries, doc_answers):
        test_case = LLMTestCase(
            input=query,
            expected_output=answer,
            actual_output=rag_chain.invoke({"input": query})["answer"],
        )
        test_cases.append(test_case)
dataset = EvaluationDataset(test_cases=test_cases)
dataset

In [ ]:
from deepeval.metrics import AnswerRelevancyMetric
from deepeval.dataset import EvaluationDataset

answer_relevancy_metric = AnswerRelevancyMetric(threshold=0.5, model=llm_judge, include_reason=True)

dataset.evaluate([answer_relevancy_metric])